In [68]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import DetCurveDisplay, RocCurveDisplay
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA

from lightgbm import LGBMRegressor
from math import sqrt
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = 10, 6

In [84]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [85]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5407 entries, 0 to 5406
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         5407 non-null   int64  
 1   CementComponent            5407 non-null   float64
 2   BlastFurnaceSlag           5407 non-null   float64
 3   FlyAshComponent            5407 non-null   float64
 4   WaterComponent             5407 non-null   float64
 5   SuperplasticizerComponent  5407 non-null   float64
 6   CoarseAggregateComponent   5407 non-null   float64
 7   FineAggregateComponent     5407 non-null   float64
 8   AgeInDays                  5407 non-null   int64  
 9   Strength                   5407 non-null   float64
dtypes: float64(8), int64(2)
memory usage: 422.5 KB


In [5]:
train

,id,CementComponent,BlastFurnaceSlag,FlyAshComponent,WaterComponent,SuperplasticizerComponent,CoarseAggregateComponent,FineAggregateComponent,AgeInDays,Strength
0,0,525.0,0.0,0.0,186.0,0.0,1125.0,613.0,3,10.38
1,1,143.0,169.0,143.0,191.0,8.0,967.0,643.0,28,23.52
2,2,289.0,134.7,0.0,185.7,0.0,1075.0,795.3,28,36.96
3,3,304.0,76.0,0.0,228.0,0.0,932.0,670.0,365,39.05
4,4,157.0,236.0,0.0,192.0,0.0,935.4,781.2,90,74.19
...,...,...,...,...,...,...,...,...,...,...
5402,5402,446.0,24.0,79.0,162.0,11.6,967.0,712.0,3,15.42
5403,5403,350.0,0.0,0.0,203.0,0.0,974.0,775.0,180,49.20
5404,5404,295.8,0.0,0.0,185.7,0.0,1076.2,759.3,28,39.30
5405,5405,376.0,93.4,0.0,162.6,11.5,955.8,662.9,28,39.61


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3605 entries, 0 to 3604
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         3605 non-null   int64  
 1   CementComponent            3605 non-null   float64
 2   BlastFurnaceSlag           3605 non-null   float64
 3   FlyAshComponent            3605 non-null   float64
 4   WaterComponent             3605 non-null   float64
 5   SuperplasticizerComponent  3605 non-null   float64
 6   CoarseAggregateComponent   3605 non-null   float64
 7   FineAggregateComponent     3605 non-null   float64
 8   AgeInDays                  3605 non-null   int64  
dtypes: float64(7), int64(2)
memory usage: 253.6 KB


In [87]:
# Split the train data into train & evl sets & 
# keep test data as test set
X = train.drop(['id', 'Strength'], axis=1)
y = train['Strength']

X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=1)

In [32]:
# Use ColumnTransformer by the column types
numeric_cols = X.select_dtypes(include='number').columns

# Fill in missing values and make them equal to the median, then standardize them
numeric_transformer = Pipeline(steps=[     
    ('imputer', SimpleImputer(strategy='median')),
])

In [33]:
X_train

,CementComponent,BlastFurnaceSlag,FlyAshComponent,WaterComponent,SuperplasticizerComponent,CoarseAggregateComponent,FineAggregateComponent,AgeInDays
1242,376.0,0.0,0.0,214.6,2.5,1003.5,762.4,100
4879,362.6,189.0,0.0,164.9,11.6,944.7,755.8,7
2652,289.0,0.0,0.0,192.0,0.0,1047.0,806.0,28
1596,135.7,203.5,0.0,185.7,0.0,1076.2,759.3,7
133,252.0,0.0,0.0,186.0,0.0,1111.0,784.0,7
...,...,...,...,...,...,...,...,...
905,252.0,0.0,0.0,185.0,0.0,1113.0,784.0,7
5192,166.1,0.0,163.3,176.5,4.5,1058.6,780.1,3
3980,298.1,0.0,108.9,209.7,8.0,838.5,815.4,28
235,305.3,203.5,0.0,203.5,0.0,965.4,695.8,3


In [34]:
# Define the second classifier (Random Forest)
RMSE_min=float('inf')

for i in range(3, 101):
    rf_clf = Pipeline(steps=[
        ('classifier', RandomForestRegressor(n_estimators=i, random_state=1))
    ])
    rf_clf.fit(X_train, y_train)
    y_pred = rf_clf.predict(X_eval)
    RMSE = sqrt(mean_squared_error(y_eval, y_pred))
    print("i:", i)
    print("RMSE:", RMSE)
    if RMSE<RMSE_min:
        RMSE_min=RMSE
        best_n_estimators=i


i: 3
RMSE: 13.750456342056387
i: 4
RMSE: 13.63029487331667
i: 5
RMSE: 13.47443854414107
i: 6
RMSE: 13.528332260248321
i: 7
RMSE: 13.479131931029292
i: 8
RMSE: 13.42842291885885
i: 9
RMSE: 13.453905087839876
i: 10
RMSE: 13.428722971124548
i: 11
RMSE: 13.369654670183364
i: 12
RMSE: 13.306881907142547
i: 13
RMSE: 13.308397864408041
i: 14
RMSE: 13.278417281001952
i: 15
RMSE: 13.270090442945662
i: 16
RMSE: 13.277821656542942
i: 17
RMSE: 13.257600324783493
i: 18
RMSE: 13.240435101129671
i: 19
RMSE: 13.23798460363279
i: 20
RMSE: 13.247492455071756
i: 21
RMSE: 13.226318639790119
i: 22
RMSE: 13.216607374266596
i: 23
RMSE: 13.198095416571878
i: 24
RMSE: 13.205195144245197
i: 25
RMSE: 13.19220084494642
i: 26
RMSE: 13.189430007119814
i: 27
RMSE: 13.175661210399868
i: 28
RMSE: 13.169137828482214
i: 29
RMSE: 13.152262629923506
i: 30
RMSE: 13.168371277065047
i: 31
RMSE: 13.175660239236114
i: 32
RMSE: 13.184047341853445
i: 33
RMSE: 13.185478102653661
i: 34
RMSE: 13.197386478802127
i: 35
RMSE: 13.18956

In [35]:
# Take i=20
rf_clf = Pipeline(steps=[
        ('classifier', RandomForestRegressor(n_estimators=20, random_state=1))
    ])
rf_clf.fit(X_train, y_train)
y_pred = rf_clf.predict(X_eval)

# Linear regression

In [36]:
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_eval)
RMSE = sqrt(mean_squared_error(y_eval, predictions))
print("RMSE:", RMSE)

RMSE: 14.811879802443942


XGBoost

In [37]:
param_grid = {
    'n_estimators': [20, 50, 100, 200, 300, 400],  # Example: search over 100, 200, and 300 trees
    'learning_rate': [0.01, 0.05, 0.1, 0.2],  # Example rates
    'max_depth': [3, 4, 5, 6, 7, 8, 9, 10]  # Common choices for tree depth
}

In [30]:
model = XGBRegressor()

# Setup grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_root_mean_squared_error', cv=5, verbose=1)
# Fit grid search
grid_search.fit(X_train, y_train)

# Best parameters and RMSE
print("Best parameters:", grid_search.best_params_)
print("Best RMSE:", -grid_search.best_score_)

Fitting 5 folds for each of 192 candidates, totalling 960 fits
Best parameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}
Best RMSE: 12.162743389641463


In [52]:
# Initialize the XGBRegressor
model = XGBRegressor(n_estimators=50, learning_rate=0.1, max_depth=3, objective='reg:squarederror', random_state=1)

# Fit the model to the training data
model.fit(X_train, y_train, eval_set=[(X_eval, y_eval)], verbose=True, early_stopping_rounds=10)
predictions = model.predict(X_eval)
rmse = sqrt(mean_squared_error(y_eval, predictions))
print(f'Root Mean Squared Error: {rmse}')

[0]	validation_0-rmse:15.56464
[1]	validation_0-rmse:15.04580
[2]	validation_0-rmse:14.60994
[3]	validation_0-rmse:14.25463
[4]	validation_0-rmse:13.95193
[5]	validation_0-rmse:13.68944
[6]	validation_0-rmse:13.47964
[7]	validation_0-rmse:13.30644
[8]	validation_0-rmse:13.15153
[9]	validation_0-rmse:13.03111
[10]	validation_0-rmse:12.92424
[11]	validation_0-rmse:12.84071
[12]	validation_0-rmse:12.76481
[13]	validation_0-rmse:12.69882
[14]	validation_0-rmse:12.65137
[15]	validation_0-rmse:12.60386
[16]	validation_0-rmse:12.56106
[17]	validation_0-rmse:12.53420
[18]	validation_0-rmse:12.51103
[19]	validation_0-rmse:12.48257
[20]	validation_0-rmse:12.46231
[21]	validation_0-rmse:12.44971
[22]	validation_0-rmse:12.43341
[23]	validation_0-rmse:12.41828
[24]	validation_0-rmse:12.41124
[25]	validation_0-rmse:12.40663
[26]	validation_0-rmse:12.39410
[27]	validation_0-rmse:12.38564
[28]	validation_0-rmse:12.38038
[29]	validation_0-rmse:12.37664
[30]	validation_0-rmse:12.37612
[31]	validation_0-

CatBoost

In [41]:
param_grid = {
    'depth': [3, 4, 6, 8, 10],
    'learning_rate': [0.01, 0.1, 0.3],
    'iterations': [100, 200, 300]
}

In [42]:
model = CatBoostRegressor(verbose=False)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_root_mean_squared_error', cv=5, verbose=1)
grid_search.fit(X_train, y_train)
print("Best parameters:", grid_search.best_params_)
print("Best RMSE:", -grid_search.best_score_)

Fitting 5 folds for each of 45 candidates, totalling 225 fits
Best parameters: {'depth': 4, 'iterations': 100, 'learning_rate': 0.1}
Best RMSE: 12.098257246517935


In [51]:
# Initialize the CatBoostRegressor
model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=4, loss_function='RMSE', verbose=100)

# Fit the model to the training data
model.fit(X_train, y_train, eval_set=(X_eval, y_eval), use_best_model=True)
predictions = model.predict(X_eval)
RMSE = sqrt(mean_squared_error(y_eval, predictions))
print("RMSE:", RMSE)

0:	learn: 15.7864834	test: 15.5736971	best: 15.5736971 (0)	total: 1.73ms	remaining: 1.73s
100:	learn: 11.6613375	test: 12.2833614	best: 12.2808628 (97)	total: 113ms	remaining: 1.01s
200:	learn: 11.3577953	test: 12.2591259	best: 12.2572039 (187)	total: 220ms	remaining: 873ms
300:	learn: 11.0863530	test: 12.2600488	best: 12.2491686 (282)	total: 313ms	remaining: 727ms
400:	learn: 10.8701091	test: 12.3082664	best: 12.2491686 (282)	total: 402ms	remaining: 600ms
500:	learn: 10.6789083	test: 12.3175200	best: 12.2491686 (282)	total: 498ms	remaining: 496ms
600:	learn: 10.5308537	test: 12.3547951	best: 12.2491686 (282)	total: 587ms	remaining: 390ms
700:	learn: 10.3995487	test: 12.3698991	best: 12.2491686 (282)	total: 679ms	remaining: 290ms
800:	learn: 10.2815584	test: 12.3926898	best: 12.2491686 (282)	total: 788ms	remaining: 196ms
900:	learn: 10.1718806	test: 12.4226733	best: 12.2491686 (282)	total: 889ms	remaining: 97.7ms
999:	learn: 10.0829767	test: 12.4592476	best: 12.2491686 (282)	total: 1s	

LightGBM

In [44]:
param_grid = {
    'num_leaves': [7, 15, 31, 62, 127, 253],
    'learning_rate': [0.01, 0.1, 0.3],
    'n_estimators': [100, 200, 300]
}

In [45]:
# Initialize the model and grid search
model = LGBMRegressor()
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_root_mean_squared_error', cv=5, verbose=1)

# Fit grid search
grid_search.fit(X_train, y_train)

# Output best parameters and RMSE
print("Best parameters:", grid_search.best_params_)
print("Best RMSE:", -grid_search.best_score_)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000193 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1070
[LightGBM] [Info] Number of data points in the train set: 3460, number of used features: 8
[LightGBM] [Info] Start training from score 35.544341
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000145 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1057
[LightGBM] [Info] Number of data points in the train set: 3460, number of used features: 8
[LightGBM] [Info] Start training from score 35.632934
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000116 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1068
[LightGBM] [Info] Number of data points in the train set: 34

In [55]:
model = LGBMRegressor(n_estimators=300, learning_rate=0.01, max_depth=-1, random_state=15)

# Fit the model to the training data
model.fit(X_train, y_train, eval_set=[(X_eval, y_eval)])
# Calculate RMSE
rmse = sqrt(mean_squared_error(y_eval, predictions))
print(f'Root Mean Squared Error: {rmse}')

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000273 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1091
[LightGBM] [Info] Number of data points in the train set: 4325, number of used features: 8
[LightGBM] [Info] Start training from score 35.531723
Root Mean Squared Error: 12.338398972853453


Now, add additional variables to check whether some of the explanatory variables are =0

In [65]:
X_train_new=X_train
X_eval_new=X_eval
X_train_new['BlastFurnaceSlagB'] = X_train_new['BlastFurnaceSlag'].apply(lambda x: 1 if x > 0 else 0)
X_train_new['FlyAshComponentB'] = X_train_new['FlyAshComponent'].apply(lambda x: 1 if x > 0 else 0)
X_train_new['SuperplasticizerComponentB'] = X_train_new['SuperplasticizerComponent'].apply(lambda x: 1 if x > 0 else 0)

X_eval_new['BlastFurnaceSlagB'] = X_eval_new['BlastFurnaceSlag'].apply(lambda x: 1 if x > 0 else 0)
X_eval_new['FlyAshComponentB'] = X_eval_new['FlyAshComponent'].apply(lambda x: 1 if x > 0 else 0)
X_eval_new['SuperplasticizerComponentB'] = X_eval_new['SuperplasticizerComponent'].apply(lambda x: 1 if x > 0 else 0)

Random forest for featured data

In [62]:
#X_train_new = X_train_new.drop(['BlastFurnaceSlag', 'FlyAshComponent', 'SuperplasticizerComponent'], axis=1)
#X_eval_new = X_eval_new.drop(['BlastFurnaceSlag', 'FlyAshComponent', 'SuperplasticizerComponent'], axis=1)

In [64]:
# Take i=20
rf_clf = Pipeline(steps=[
    ('classifier', RandomForestRegressor(n_estimators=20, random_state=1))
])
rf_clf.fit(X_train_new, y_train)
y_pred = rf_clf.predict(X_eval_new)
RMSE = sqrt(mean_squared_error(y_eval, y_pred))
print("RMSE:", RMSE)

RMSE: 13.285005348202276


CatBoost for featured data

In [66]:
# Initialize the CatBoostRegressor
model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=4, loss_function='RMSE', verbose=100)

# Fit the model to the training data
model.fit(X_train_new, y_train, eval_set=(X_eval_new, y_eval), use_best_model=True)
predictions = model.predict(X_eval_new)
RMSE = sqrt(mean_squared_error(y_eval, predictions))
print("RMSE:", RMSE)

0:	learn: 15.8151199	test: 15.5905139	best: 15.5905139 (0)	total: 1.6ms	remaining: 1.6s
100:	learn: 11.6710000	test: 12.3055610	best: 12.3015407 (88)	total: 113ms	remaining: 1.01s
200:	learn: 11.3509851	test: 12.2886179	best: 12.2795960 (156)	total: 218ms	remaining: 866ms
300:	learn: 11.1037107	test: 12.2773876	best: 12.2736648 (295)	total: 320ms	remaining: 743ms
400:	learn: 10.8914790	test: 12.3241844	best: 12.2736648 (295)	total: 416ms	remaining: 621ms
500:	learn: 10.7039006	test: 12.3610914	best: 12.2736648 (295)	total: 520ms	remaining: 517ms
600:	learn: 10.5462457	test: 12.3896829	best: 12.2736648 (295)	total: 625ms	remaining: 415ms
700:	learn: 10.4020709	test: 12.4202310	best: 12.2736648 (295)	total: 727ms	remaining: 310ms
800:	learn: 10.2828707	test: 12.4158881	best: 12.2736648 (295)	total: 826ms	remaining: 205ms
900:	learn: 10.1752219	test: 12.4593804	best: 12.2736648 (295)	total: 935ms	remaining: 103ms
999:	learn: 10.0801020	test: 12.4739741	best: 12.2736648 (295)	total: 1.05s	

## PCA

In [80]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
pca = PCA(n_components=0.95)  # Keep 95% of variance
X_train_pca = pca.fit_transform(X_train_scaled)

print(f"Original number of features: {X_train_scaled.shape[1]}")
print(f"Reduced number of features: {X_train_pca.shape[1]}")

Original number of features: 11
Reduced number of features: 7


In [72]:
X_eval_scaled = scaler.transform(X_eval)
X_eval_pca = pca.transform(X_eval_scaled)

### Random Forest

In [76]:
# Take i=20
rf_clf = Pipeline(steps=[
    ('classifier', RandomForestRegressor(n_estimators=15, random_state=1))
])
rf_clf.fit(X_train_pca, y_train)
y_pred = rf_clf.predict(X_eval_pca)
RMSE = sqrt(mean_squared_error(y_eval, y_pred))
print("RMSE:", RMSE)

RMSE: 13.254319806592504


In [24]:
# Split the train data into train & evl sets & 
# keep test data as test set
X_test = test.drop(['id', 'Strength'], axis=1)
y_test = test['Strength']

rf_clf.fit(X_test, y_test)
y_pred = rf_clf.predict(X_test)

KeyError: "['Strength'] not found in axis"

In [88]:
# Initialize the CatBoostRegressor
model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=4, loss_function='RMSE', verbose=100)

# Fit the model to the training data
model.fit(X_train, y_train, eval_set=(X_eval, y_eval), use_best_model=True)
predictions = model.predict(X_eval)
RMSE = sqrt(mean_squared_error(y_eval, predictions))
print("RMSE:", RMSE)

0:	learn: 15.7864834	test: 15.5736971	best: 15.5736971 (0)	total: 1.49ms	remaining: 1.48s
100:	learn: 11.6613375	test: 12.2833614	best: 12.2808628 (97)	total: 117ms	remaining: 1.04s
200:	learn: 11.3577953	test: 12.2591259	best: 12.2572039 (187)	total: 217ms	remaining: 864ms
300:	learn: 11.0863530	test: 12.2600488	best: 12.2491686 (282)	total: 313ms	remaining: 726ms
400:	learn: 10.8701091	test: 12.3082664	best: 12.2491686 (282)	total: 402ms	remaining: 600ms
500:	learn: 10.6789083	test: 12.3175200	best: 12.2491686 (282)	total: 495ms	remaining: 493ms
600:	learn: 10.5308537	test: 12.3547951	best: 12.2491686 (282)	total: 589ms	remaining: 391ms
700:	learn: 10.3995487	test: 12.3698991	best: 12.2491686 (282)	total: 679ms	remaining: 290ms
800:	learn: 10.2815584	test: 12.3926898	best: 12.2491686 (282)	total: 773ms	remaining: 192ms
900:	learn: 10.1718806	test: 12.4226733	best: 12.2491686 (282)	total: 868ms	remaining: 95.4ms
999:	learn: 10.0829767	test: 12.4592476	best: 12.2491686 (282)	total: 973

In [89]:
X_test = test.drop(['id'], axis=1) 

test_predict = model.predict(X_test) # tpr = 1

submission_df = pd.DataFrame({'id': test['id'], 
                            'Strength': test_predict})
submission_df

,id,Strength
0,5407,49.094380
1,5408,18.560920
2,5409,34.358290
3,5410,49.061973
4,5411,30.869753
...,...,...
3600,9007,29.011628
3601,9008,35.628721
3602,9009,39.777208
3603,9010,35.307123


In [90]:
submission_df.to_csv('submission.csv', index=None)